__In ngsLD conda env__

In [ ]:
conda create --name ngsld

__follow install instructions https://github.com/fgvieira/ngsLD__

In [44]:
%cd /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/ngsld/

/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/ngsld


__check number of positions- subtract 1 for header row__

In [21]:
!zcat ensembl_appc_angsd_probs_doGeno8.beagle.gz | wc -l
#!cat appc_angsd_pos.txt | wc -l

6295268


In [28]:
%cd /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd

/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd


__make a positions file for ngsld__


In [41]:
!zcat angsd/ensembl_appc_angsd_probs.maf.gz | awk '{print $1"\t"$2}' > angsd/ngsld/ensembl_appc_angsd_probs_doGeno8_forngsLD_pos.txt #needs to be tab separated

In [42]:
!head angsd/ngsld/ensembl_appc_angsd_probs_doGeno8_forngsLD_pos.txt 

Chromo	Position
1	164
1	479
1	480
1	787
1	1492
1	2973
1	3230
1	3234
1	3335


__this works, from ~/ in base environment or ngsld in terminal__ 


In [ ]:
nohup /local/home/katrinac/ngsLD/ngsLD --geno /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/ensembl_appc_angsd_probs_doGeno8.beagle.gz --probs --n_ind 159 --n_threads 40 --n_sites 6295267 --posH /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/ngsld/ensembl_appc_angsd_probs_doGeno8_forngsLD_pos.txt --outH /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/ngsld/ensembl_appc_angsd_probs_doGeno8_ngsLD.tsv >> nohup_ngsld.out &

__fit LD decay model- in terminal, be sure to move to background__

__file is too big, need to randomly sample first__

In [45]:
!shuf -n 10000 /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/appc_ngsLD.tsv > /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/appc_ngsLD_sample.tsv

In [ ]:
#add the header line
!sed -i '1 i #site1\tsite2\tdist\tr^2_ExpG\tD\tD'\tr^2' /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/appc_ngsLD_sample.ld

In [49]:
!tail /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/appc_ngsLD.tsv
!tail /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/appc_ngsLD_sample.tsv

PYAM01000117.1:3846	PYAM01000117.1:10455	6609	0.009559	0.067509	0.481028	0.190643
PYAM01000117.1:4443	PYAM01000117.1:22287	17844	0.000683	-0.039773	0.576455	0.049491
PYAM01000117.1:3854	PYAM01000117.1:22287	18433	0.008007	-0.047751	0.957737	0.079045
PYAM01000117.1:3882	PYAM01000117.1:22287	18405	0.022576	-0.023291	0.753534	0.026008
PYAM01000117.1:2580	PYAM01000117.1:10445	7865	0.021801	-0.008679	0.976184	0.011390
PYAM01000117.1:3846	PYAM01000117.1:20475	16629	0.000472	0.020349	0.185111	0.009956
PYAM01000117.1:2580	PYAM01000117.1:10455	7875	0.016249	-0.006922	0.978035	0.008570
PYAM01000117.1:3846	PYAM01000117.1:22287	18441	0.005994	-0.021501	0.669912	0.021539
PYAM01000117.1:2580	PYAM01000117.1:20475	17895	0.027384	-0.015047	0.764707	0.022975
PYAM01000117.1:2580	PYAM01000117.1:22287	19707	0.013888	0.023868	0.722273	0.124846
CM009719.1:7049360	CM009719.1:7077477	28117	0.047725	0.122486	0.742718	0.458564
CM009730.1:4527358	CM009730.1:4578672	51314	0.000020	-0.016891	0.762993	0.018267
CM009

In [37]:
Rscript --vanilla --slave scripts/fit_LDdecay.R --ld_files /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/infiles.ld --plot_x_lim 14 --n_ind 159 --ld r2 --max_kb_dist 200 --fit_boot 100 --fit_bin_size 1000 --fit_level 100 --plot_data --plot_scale 3 --out /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/LD_plot_CI.pdf

UsageError: Line magic function `%` not found.


__the pruning perl script takes a long time and is not possible to multi thread so split file by chromosome- use R to create a chromosomes file from the maf with a data.table summary, but group PYAM together so there aren't hundreds of files because those are contigs__

In [ ]:
!for i in $(cat /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/ngsld/chromosomes.txt ); do echo -e "#site1\tsite2\tdist\tr^2_ExpG\tD\tD'\tr^2" > /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/$i\_appc_ngsLD.tsv; grep "^$i" /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/ngsld/appc_ngsLD.tsv >> /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/genotypes/angsd/ngsld/$i\_appc_ngsLD.tsv; done

__execute the perl script below for each chrom ld file made above__

In [ ]:
nohup perl /local/home/katrinac/ngsLD/scripts/prune_graph.pl --in_file /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/appc_ngsLD.tsv --max_kb_dist 5 --min_weight 0.5 --out /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/angsd/ngsld/appc_unlinked.id >> nohup_perl_ld.out &

__cat the files together for one unlinked text file for angsd__

### move back to ANGSD_genolike.ipynb to restrict to the unlinked sites